In [1]:
import pandas as pd
import pickle

In [ ]:
import sys
print(sys.executable)


In [2]:
file_path = '../../Obstetric accessibility visualization/Data/Cleaned_Block_Group_Pop_Centers.csv'
BG_pop_centers = pd.read_csv(file_path)
BG_pop_centers.head()

,FIPS,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,POPULATION,LATITUDE,LONGITUDE,Coordinates,geometry,Geography
0,19001,19,1,960100,1,1374,41.458179,-94.342516,"(-94.342516, 41.458179)",POINT (-94.342516 41.458179),1500000US190019601001
1,19001,19,1,960100,2,613,41.438395,-94.575000,"(-94.575, 41.438395)",POINT (-94.575 41.438395),1500000US190019601002
2,19001,19,1,960100,3,789,41.500753,-94.646195,"(-94.646195, 41.500753)",POINT (-94.646195 41.500753),1500000US190019601003
3,19001,19,1,960200,1,550,41.261736,-94.338260,"(-94.33826, 41.261736)",POINT (-94.33826 41.261736),1500000US190019602001
4,19001,19,1,960200,2,1061,41.221432,-94.524858,"(-94.524858, 41.221432)",POINT (-94.524858 41.221432),1500000US190019602002


In [3]:
file_path = '../../Obstetric accessibility visualization/Data/Iowa Hospitals - Iowa hospitals.csv'
Hospital_cd=pd.read_csv(file_path)
Hospital_cd.head()

,Hospital Level,Hospital,Perinatal Level,County,Region,Address,Coordinates,CCO,NICU,ICU
0,Level 1,"Kossuth Regional Health Center, Algona (Kossu...",1,Kossuth,Algona,"1515 S Phillips St, Algona, IA 50511","43.056298091772184, -94.22589193539095",0,0,0
1,Level 1,"Cass County Memorial Hospital, Atlantic (Cass...",1,Cass,Atlantic,"1501 E 10th St, Atlantic, IA 50022","41.40142135760074, -94.99082668703227",0,0,0
2,Level 1,"Boone County Hospital, Boone (Boone County)",1,Boone,Boone,"1015 Union St, Boone, IA 50036","42.06861215749349, -93.87320920562937",0,0,0
3,Level 1,"St. Anthony Regional Hospital, Carroll (Carrol...",1,Carroll,Carroll,"311 S Clark St, Carroll, IA 51401","42.05682070761609, -94.86715192582066",0,0,0
4,Level 1,"Floyd County Memorial Hospital, Charles City ...",1,Floyd,Charles City,"1501 S Main St, Charles City, IA 50616","43.051140682166505, -92.68415484205565",0,0,0


In [ ]:
import geopandas as gpd
from shapely.geometry import Point


In [4]:
Hospital_cd[['Latitude', 'Longitude']] = Hospital_cd['Coordinates'].str.split(', ', expand=True)

In [ ]:
# Hospital_cd['Latitude'] = None
# Hospital_cd['Longitude'] = None

# Hospital_cd[['Latitude', 'Longitude']] = Hospital_cd['Coordinates'].str.split(', ', expand=True)

In [5]:
Hospital_cd = Hospital_cd.drop(columns=['Coordinates'])

In [6]:
Hospital_cd.head()

,Hospital Level,Hospital,Perinatal Level,County,Region,Address,CCO,NICU,ICU,Latitude,Longitude
0,Level 1,"Kossuth Regional Health Center, Algona (Kossu...",1,Kossuth,Algona,"1515 S Phillips St, Algona, IA 50511",0,0,0,43.056298091772184,-94.22589193539095
1,Level 1,"Cass County Memorial Hospital, Atlantic (Cass...",1,Cass,Atlantic,"1501 E 10th St, Atlantic, IA 50022",0,0,0,41.40142135760074,-94.99082668703227
2,Level 1,"Boone County Hospital, Boone (Boone County)",1,Boone,Boone,"1015 Union St, Boone, IA 50036",0,0,0,42.06861215749349,-93.87320920562937
3,Level 1,"St. Anthony Regional Hospital, Carroll (Carrol...",1,Carroll,Carroll,"311 S Clark St, Carroll, IA 51401",0,0,0,42.05682070761609,-94.86715192582066
4,Level 1,"Floyd County Memorial Hospital, Charles City ...",1,Floyd,Charles City,"1501 S Main St, Charles City, IA 50616",0,0,0,43.051140682166505,-92.68415484205565


In [7]:
BG_pop_centers.head()\


,FIPS,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,POPULATION,LATITUDE,LONGITUDE,Coordinates,geometry,Geography
0,19001,19,1,960100,1,1374,41.458179,-94.342516,"(-94.342516, 41.458179)",POINT (-94.342516 41.458179),1500000US190019601001
1,19001,19,1,960100,2,613,41.438395,-94.575000,"(-94.575, 41.438395)",POINT (-94.575 41.438395),1500000US190019601002
2,19001,19,1,960100,3,789,41.500753,-94.646195,"(-94.646195, 41.500753)",POINT (-94.646195 41.500753),1500000US190019601003
3,19001,19,1,960200,1,550,41.261736,-94.338260,"(-94.33826, 41.261736)",POINT (-94.33826 41.261736),1500000US190019602001
4,19001,19,1,960200,2,1061,41.221432,-94.524858,"(-94.524858, 41.221432)",POINT (-94.524858 41.221432),1500000US190019602002


In [ ]:
alpha = {}
for b in BG_pop_centers.index:
    alpha[b] = {}
    for h in Hospital_cd.index:
        b_coords = (BG_pop_centers.loc[b]['LATITUDE'],BG_pop_centers.loc[b]['LONGITUDE'])
        h_coords = (Hospital_cd.loc[h]['Latitude'], Hospital_cd.loc[h]['Longitude'])
        dist = geopy.distance.great_circle(b_coords,h_coords).miles
        alpha[b][h]=dist

In [ ]:
import requests

def get_osrm_road_distance(start_coords, end_coords):
    # Prepare the URL for the OSRM API
    osrm_url = f"http://router.project-osrm.org/route/v1/driving/{start_coords[1]},{start_coords[0]};{end_coords[1]},{end_coords[0]}?overview=false"
    
    # Send a request to the OSRM API
    response = requests.get(osrm_url)
    data = response.json()
    
    # Extract the distance from the JSON response
    # Note: OSRM returns distance in meters, so we convert it to miles
    distance_meters = data['routes'][0]['distance']
    distance_miles = distance_meters / 1609.34  # Convert meters to miles
    
    return distance_miles

# Assuming BG_pop_centers and Hospital_cd are your dataframes with latitude and longitude information
alpha = {}
for b in BG_pop_centers.index:
    alpha[b] = {}
    for h in Hospital_cd.index:
        b_coords = (BG_pop_centers.loc[b]['LATITUDE'], BG_pop_centers.loc[b]['LONGITUDE'])
        h_coords = (Hospital_cd.loc[h]['Latitude'], Hospital_cd.loc[h]['Longitude'])
        
        # Use the OSRM API to get road distance in miles
        dist = get_osrm_road_distance(b_coords, h_coords)
        
        alpha[b][h] = dist


In [ ]:
import requests

def get_graphhopper_road_distance(start_coords, end_coords, api_key):
    # Format the coordinates
    start_str = f"{start_coords[1]},{start_coords[0]}"  # GraphHopper expects lon,lat
    end_str = f"{end_coords[1]},{end_coords[0]}"
    
    # Prepare the URL for the GraphHopper Routing API
    url = f"https://graphhopper.com/api/1/route?point={start_str}&point={end_str}&vehicle=car&locale=en&key={api_key}"
    
    # Send a request to the GraphHopper API
    response = requests.get(url)
    data = response.json()
    
    # Extract the distance from the JSON response (in meters) and convert to miles
    distance_meters = data['paths'][0]['distance']
    distance_miles = distance_meters / 1609.34  # Convert meters to miles
    
    return distance_miles

# Example usage
api_key = "3aefeeef-2bfb-4792-9298-bd636c5c00ad"
alpha = {}

for b in BG_pop_centers.index:
    alpha[b] = {}
    for h in Hospital_cd.index:
        b_coords = (BG_pop_centers.loc[b]['LATITUDE'], BG_pop_centers.loc[b]['LONGITUDE'])
        h_coords = (Hospital_cd.loc[h]['Latitude'], Hospital_cd.loc[h]['Longitude'])
        
        # Use the GraphHopper API to get road distance in miles
        dist = get_graphhopper_road_distance(b_coords, h_coords, api_key)
        
        alpha[b][h] = dist


In [ ]:
import requests
import time

def get_ors_road_distance(start_coords, end_coords, api_key):
    url = f"https://api.openrouteservice.org/v2/directions/driving-car?start={start_coords[1]},{start_coords[0]}&end={end_coords[1]},{end_coords[0]}"
    headers = {"Authorization": api_key}
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        distance_meters = data['features'][0]['properties']['segments'][0]['distance']
        distance_miles = distance_meters / 1609.34  # Convert meters to miles
        return distance_miles
    else:
        print(f"Error fetching data: HTTP {response.status_code} - {response.text}")
        return None

api_key = "5b3ce3597851110001cf62481dbafedbe9544cd9b3723b8ecbdf1ac4"

alpha = {}
request_count = 0
start_time = time.time()

for b in BG_pop_centers.index:
    alpha[b] = {}
    for h in Hospital_cd.index:
        if request_count >= 40:
            # If 60 seconds haven't passed since we started, wait until they do
            while time.time() - start_time < 60:
                time.sleep(1)  # Sleep for a bit before checking the time again
            
            # Reset the counter and the timer
            request_count = 0
            start_time = time.time()
            
        b_coords = (BG_pop_centers.loc[b]['LATITUDE'], BG_pop_centers.loc[b]['LONGITUDE'])
        h_coords = (Hospital_cd.loc[h]['Latitude'], Hospital_cd.loc[h]['Longitude'])
        
        try:
            dist = get_ors_road_distance(b_coords, h_coords, api_key)
            if dist is not None:
                alpha[b][h] = dist
            else:
                alpha[b][h] = 'Distance calculation failed'
        except Exception as e:
            print(f"Error calculating distance from {b} to {h}: {e}")
            alpha[b][h] = 'Error'
        
        request_count += 1
        time.sleep(1.5)  # Adjusted sleep time for an average distribution of requests


In [ ]:
import requests
import time

def get_ors_road_distance(start_coords, end_coords, api_key):
    url = f"https://api.openrouteservice.org/v2/directions/driving-car?start={start_coords[1]},{start_coords[0]}&end={end_coords[1]},{end_coords[0]}"
    headers = {"Authorization": api_key}
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        distance_meters = data['features'][0]['properties']['segments'][0]['distance']
        distance_miles = distance_meters / 1609.34  # Convert meters to miles
        return distance_miles
    else:
        print(f"Error fetching data: HTTP {response.status_code} - {response.text}")
        return None

api_key = "5b3ce3597851110001cf62481dbafedbe9544cd9b3723b8ecbdf1ac4"

alpha = {}
request_count = 0
start_time = time.time()

# Assuming BG_pop_centers and Hospital_cd are pandas DataFrames with 'LATITUDE' and 'LONGITUDE' columns
# Adjust the loop to only process one hospital and a fixed number of BG coordinates

# Select the first hospital for demonstration
first_hospital_index = Hospital_cd.index[0]
h_coords = (Hospital_cd.loc[first_hospital_index]['Latitude'], Hospital_cd.loc[first_hospital_index]['Longitude'])

# Process only the first 4 BG coordinates
for b in BG_pop_centers.index[:4]:
    if request_count >= 40:
        # If 60 seconds haven't passed since we started, wait until they do
        while time.time() - start_time < 60:
            time.sleep(1)  # Sleep for a bit before checking the time again
        
        # Reset the counter and the timer
        request_count = 0
        start_time = time.time()
        
    b_coords = (BG_pop_centers.loc[b]['LATITUDE'], BG_pop_centers.loc[b]['LONGITUDE'])
    
    try:
        dist = get_ors_road_distance(b_coords, h_coords, api_key)
        if dist is not None:
            alpha[b] = {first_hospital_index: dist}
        else:
            alpha[b] = {first_hospital_index: 'Distance calculation failed'}
    except Exception as e:
        print(f"Error calculating distance from {b} to {first_hospital_index}: {e}")
        alpha[b] = {first_hospital_index: 'Error'}
    
    request_count += 1
    time.sleep(1.5)  # Adjusted sleep time for an average distribution of requests

# Print or break after the first 4 BG coordinates are processed
print("Completed distance calculations for the first hospital and first 4 BG coordinates.")
# No need for an explicit break since the loop naturally ends after 4 iterations


In [ ]:
import requests

def get_ors_road_distance(start_coords, end_coords, api_key):
    url = f"https://api.openrouteservice.org/v2/directions/driving-car?start={start_coords[1]},{start_coords[0]}&end={end_coords[1]},{end_coords[0]}"
    headers = {"Authorization": api_key}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        distance_meters = data['features'][0]['properties']['segments'][0]['distance']
        distance_miles = distance_meters / 1609.34
        return distance_miles
    else:
        print(f"Error fetching data: HTTP {response.status_code} - {response.text}")
        return None

# Replace "YOUR_API_KEY" with your actual OpenRouteService API key
api_key = "5b3ce3597851110001cf62481dbafedbe9544cd9b3723b8ecbdf1ac4"

# Define two sample sets of coordinates (latitude, longitude)
start_coords = (41.458179, -94.342516) 	 
end_coords = (43.056298091772184, -94.22589193539095) 

# Attempt to calculate the road distance between the two points
distance = get_ors_road_distance(start_coords, end_coords, api_key)
if distance is not None:
    print(f"Road distance: {distance} miles")
else:
    print("Failed to calculate distance.")


In [ ]:
# save dictionary
with open('../../Obstetric accessibility visualization/Data/Distance_Matrix.pkl', 'wb') as fp:
    pickle.dump(alpha, fp)

In [ ]:
with open('../../Obstetric accessibility visualization/Data/Distance_Matrix.pkl', 'rb') as file:
    df = pickle.load(file)

In [ ]:
df

In [ ]:
# from shapely.geometry import Point

# Hospital_cd['Coordinates'] = Hospital_cd.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
# geometry = [Point(xy) for xy in zip(Hospital_cd['Coordinates'])]
# Hospital_cd = gpd.GeoDataFrame(Hospital_cd, 
#                         crs = iowa_BG.crs,
#                         geometry = geometry)
Hospital_cd['Coordinates'] = Hospital_cd.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)

In [ ]:
pip install geopy

In [ ]:
Hospital_cd.head()


In [ ]:
from math import radians, sin, cos, sqrt, atan2

# Assuming you have two GeoDataFrames: Hospital_cd and BG_pop_centers
# Create an empty DataFrame to store the distance matrix
distance_matrix = pd.DataFrame(index=Hospital_cd['Hospital'], columns=BG_pop_centers['Geography'])

# Define the Haversine formula function
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c  # Radius of Earth in kilometers
    return distance

# Loop through hospitals and block groups and calculate distances
for hospital_idx, hospital_row in Hospital_cd.iterrows():
    for block_group_idx, block_group_row in BG_pop_centers.iterrows():
        # Extract latitude and longitude from the 'Point' objects
        lat1, lon1 = hospital_row['Coordinates'].x, hospital_row['Coordinates'].y
        lat2, lon2 = block_group_row['geometry'].x, block_group_row['geometry'].y
        # Calculate distance using Haversine formula
        distance = haversine(lat1, lon1, lat2, lon2)
        # Store the distance in the distance matrix
        distance_matrix.at[hospital_row['Hospital'], block_group_row['Geography']] = distance

# Convert the distance matrix to a numeric type (float) and fill any missing values
distance_matrix = distance_matrix.apply(pd.to_numeric, errors='coerce')

In [ ]:
distance_matrix

In [ ]:
csv_file_path = '../../Obstetric accessibility visualization/Data/distance_matrix.csv'

# Save the distance matrix to a CSV file
distance_matrix.to_csv(csv_file_path)

In [ ]:
nan_mask = distance_matrix >50

# Use the mask to filter the distance_matrix
filtered_distance_matrix = distance_matrix[nan_mask]
filtered_distance_matrix